In [ ]:
import itertools
import random
from enum import Enum
from functools import cache

import numpy as np

## Tipos de datos

In [ ]:
# Tipos de datos
Board = np.ndarray
Vector = tuple[int, int]
Cell = Vector
Tile = int

## Direcciones

Para hacer el código más legible, se ha creado la clase de tipo enumeración Direction. Así se pueden utilizar expresiones cómo Direction.UP o Direction.RIGHT.

In [ ]:
class Direction(Enum):
    UP = (-1, 0)
    RIGHT = (0, 1)
    DOWN = (1, 0)
    LEFT = (0, -1)

    @property
    def vector(self) -> Vector:
        """
        Simplemente para poder obtener el vector como si fuese un atributo:
        Direction.UP.vector
        direction.vector
        :return: Valor(vector) almacenado en el elemento.
        """
        return self.value

## Funciones generales

In [ ]:
def create_board() -> Board:
    """
    Obtiene un nuevo tablero vacío.
    :return: Tablero vacío.
    """
    return np.zeros(shape=(Game.rows, Game.cols), dtype=np.int16)


def clone_board(board: Board) -> Board:
    """
    Obtiene un clon del tablero dado.
    :param board: Tablero a clonar.
    :return: Clon del tablero.
    """
    return np.copy(board)


def add_tile(board: Board, cell: Cell, tile: Tile) -> Board:
    """
    Obtiene un tablero nuevo con la baldosa añadida.
    :param board: Tablero sobre el que añadir la baldosa.
    :param cell: Celda en la que colocar la baldosa.
    :param tile: Baldosa a colocar.
    :return: Tablero con la baldosa colocada.
    """
    board = clone_board(board)
    board[cell] = tile
    return board


def in_board(cell: Cell) -> bool:
    """
    Comprueba si la celda dada pertenece al tablero.
    :param cell: Celda a comprobar.
    :return: Si la celda pertenece al tablero.
    """
    return 0 <= cell[0] < Game.rows and 0 <= cell[1] < Game.cols


def sum_vectors(vector: Vector, other: Vector) -> Vector:
    """
    Suma 2 vectores de 2 dimensiones.
    :param vector: Vector a sumar.
    :param other: Vector a suman.
    :return: Suma de los vectores.
    """
    return vector[0] + other[0], vector[1] + other[1]


@cache
def get_board_cells(direction=Direction.UP) -> tuple[Cell]:
    """
    Obtiene una tupla de celdas del tablero, ordenadas según la dirección en la
    que se intente realizar un movimiento.
    :param direction: Dirección de movimiento.
    :return: Tupla de celdas ordenadas.
    """
    # Establecer el orden de recorrido atendiendo a la dirección.
    row_order = range(Game.rows) if direction != Direction.DOWN else range(Game.rows - 1, -1, -1)
    col_order = range(Game.cols) if direction != Direction.RIGHT else range(Game.cols - 1, -1, -1)

    # Producto cartesiano entre las filas y columnas ordenadas.
    iterator = itertools.product(row_order, col_order)

    # Convertir el iterador a una tupla para poder almacenarlo en cache.
    return tuple(iterator)


def get_possible_moves(board: Board) -> list[Direction]:
    """
    Obtiene las posibles direcciones en las que se puede mover el tablero.
    :param board: Tablero.
    :return: Direcciones posibles.
    """
    return list(filter(lambda d: can_move_direction(board, d), Direction))

## Añadir baldosa aleatoria

In [ ]:
def get_empty_cells(board: Board) -> list[Cell]:
    """
    Obtiene una lista con las celdas vacías del tablero.
    :param board: Tablero.
    :return: Celdas vacías del tablero.
    """
    return [tuple(cell) for cell in np.argwhere(board == 0)]


def get_next_cell(board: Board) -> Cell:
    """
    Obtiene una celda vacía (pseudo)aleatoria del tablero dado.
    :param board: Tablero.
    :return: Celda vacía aleatoria.
    """
    return random.choice(get_empty_cells(board))


def get_next_tile() -> Tile:
    """
    Obtiene una baldosa nueva a colocar atendiendo a las probabilidades
    dadas en la clase Game.
    :return: Siguiente baldosa a colocar.
    """
    # random.choices devuelve una lista.
    return random.choices(Game.new_tile_tile, Game.new_tile_prob)[0]


def add_random_tile(board: Board):
    """
    Añade una baldosa de forma aleatoria al tablero dado.
    :param board: Tablero
    """
    cell = get_next_cell(board)
    tile = get_next_tile()
    board[cell] = tile

## Realizar movimiento

In [ ]:

def get_last_and_next_cell(board: Board, last_cell: Cell, direction: Direction) -> tuple[Cell, Cell]:
    """
    Obtiene la última celda vacía a la que se puede llegar desde la celda dada, en el tablero
    y dirección dados, junto con la celda inmediatamente posterior a esta.
    :param board: Tablero.
    :param last_cell: Celda inicial.
    :param direction: Dirección del movimiento.
    :return: Celdas última y siguiente .
    """
    next_cell = sum_vectors(last_cell, direction.vector)
    while in_board(next_cell) and not board[next_cell]:
        last_cell = next_cell
        next_cell = sum_vectors(last_cell, direction.vector)
    return last_cell, next_cell


def move(board: Board, direction: Direction) -> Board:
    """
    Obtiene un tablero resultado de mover el tablero dado en la dirección dada.
    :param board: Tablero sobre el que realizar el movimiento.
    :param direction: Dirección en la que mover.
    :return: Tablero movido.
    """
    moved = create_board()
    # Conjunto ya que facilita la consulta de pertenencia.
    merged = set()

    for original_cell in get_board_cells(direction):
        tile = board[original_cell]
        if not tile:
            continue

        last_cell, next_cell = get_last_and_next_cell(moved, original_cell, direction)

        if in_board(next_cell) and tile == moved[next_cell] and next_cell not in merged:
            # Combinar
            moved[next_cell] = 2 * tile
            merged.add(next_cell)
        else:
            # Mover
            moved[last_cell] = tile

    return moved

## Comprobaciones

In [ ]:
def can_move_direction(board: Board, direction: Direction) -> bool:
    """
    Comprueba se el tablero dado se puede mover en la dirección dada.
    :param board: Tablero.
    :param direction: Dirección.
    :return: Si se puede mover el tablero en la dirección.
    """
    for cell, tile in np.ndenumerate(board):
        if not tile:
            continue
        next_cell = sum_vectors(cell, direction.vector)
        if in_board(next_cell):
            next_tile = board[next_cell]
            if not next_tile or tile == next_tile:
                return True

    return False


def can_move(board: Board) -> bool:
    """
    Comprueba si se puede mover el tablero dado en alguna dirección.
    :param board: Tablero.
    :return: Si se puede mover.
    """
    for direction in Direction:
        if can_move_direction(board, direction):
            return True
    return False


def has_lost(board: Board) -> bool:
    """
    Comprueba si el tablero dado se corresponde con una partida perdida.
    :param board: Tablero.
    :return: Si se ha perdido la partida.
    """
    return not can_move(board)


def has_won(board: Board) -> bool:
    """
    Comprueba si el tablero dado se corresponde con una victoria.
    :param board: Tablero.
    :return: Si se ha ganado la partida.
    """
    for tile in board.flat:
        if tile == 2048:
            return True
    return False

def has_finished(board: Board) -> bool:
    return has_lost(board) or has_won(board)

## Clase Juego Básica

In [ ]:
class Game:
    rows = 4
    cols = 4
    initial_tiles = 2
    new_tile_tile = [2, 4]
    new_tile_prob = [90, 10]

    def __init__(self):
        """
        Acciones que se realizan al iniciar el juego.
        """
        self.board = create_board()
        [add_random_tile(self.board) for _ in range(self.initial_tiles)]

    def move(self, direction: Direction):
        """
        Realizar un movimiento.
        """
        moved = move(self.board, direction)
        if not np.array_equal(self.board, moved):
            self.board = moved
            add_random_tile(self.board)